In [1]:
import casadi as cas
import numpy as np
import matplotlib.pyplot as plt

## Ejercicio 1 parte 1

In [2]:
def g(w):
    x = w[0]
    y = w[1]
    g1 = cas.tanh((x+2)*y**2/25 - 0.5)
    g2 = cas.sin(x) - 0.5*y + 1
    return cas.vertcat(g1, g2)

In [3]:
g(cas.vertcat(-0.8, 2))

DM([-0.298617, -0.717356])

In [4]:
g(np.array([-0.8, 2]))

DM([-0.298617, -0.717356])

In [5]:
def forw_jac_col(g, x, col, eps = 1e-8):
    vec_l = len(x)
    fw_seed = np.zeros_like(x)
    fw_seed[col] = 1
    return(g(x + eps*fw_seed)- g(x))/eps

In [6]:
x_0 = np.array([-0.8, 2])

In [7]:
forw_jac_col(g, x_0, 0)

DM([0.145733, 0.696707])

In [8]:
# Mejorar con *cols
def jacobian(g, x):
    jac = forw_jac_col(g, x, 0)
    for col in range(1,len(x)):
        jac = cas.horzcat(jac, forw_jac_col(g, x, col))
    return jac

In [9]:
jacobian(g, x_0)

DM(
[[0.145733, 0.174879], 
 [0.696707, -0.5]])

In [10]:
def delta_x(g, x):
    jac = jacobian(g, x)
    return cas.solve(-jac, g(x))

In [11]:
def step (g, x):
    dx = delta_x(g,x)
    return np.array(x + dx)

In [12]:
x = x_0
for i in range(5):
    x = step(g, x)
    print(x)

[[0.61114968]
 [2.53160267]]
[[0.16714937]
 [2.42035621]]
[[0.19440912]
 [2.38650392]]
[[0.19453578]
 [2.38662219]]
[[0.19453578]
 [2.38662219]]


## Ejercicio 2 partes 1 a 3

In [13]:
x = cas.MX.sym('x')
y = cas.MX.sym('y')
p = cas.MX.sym('p',2)
A = cas.MX.sym('A', 4, 5)
f = cas.Function('f', [x,y], [cas.sin(x) * y])
g = cas.Function('g', [x], [cas.sqrt(x), x**2])
h = cas.Function('h', [x], [cas.sin(x)*y])
F = lambda x,y: cas.sin(x) * y

In [14]:
cas.Function('m', [x], [cas.sin(x)])

Function(m:(i0)->(o0) MXFunction)

In [15]:
cas.vertcat(x,y)

MX(vertcat(x, y))

In [16]:
h(x)**2

MX(sq((sin(x)*y)))

In [17]:
cas.Function('f2', [x], [h(x)**2])

Function(f2:(i0)->(o0) MXFunction)

In [18]:
h(cas.MX(5))

MX((-0.958924*y))

In [19]:
cas.Function('m', [x,y], [x])

Function(m:(i0,i1)->(o0) MXFunction)

In [20]:
f2 = cas.Function('f2', [x,y], [cas.sin(x)+y], ['x','y'], ['z'])

In [21]:
f2(x = 3)

{'z': DM(0.14112)}

In [22]:
f2()

{'z': DM(0)}

In [23]:
g2 = cas.Function('g2', [x], [cas.sqrt(x), x**2], ['x'], ['p0', 'p1'])

In [24]:
g2(x=3)

{'p0': DM(1.73205), 'p1': DM(9)}

In [25]:
g2([])

(DM(0), DM(0))

## Ejercicio 1 parte 2

In [26]:
def g(w):
    x = w[0]
    y = w[1]
    g1 = cas.tanh((x+2)*y**2/25 - 0.5)
    g2 = cas.sin(x) - 0.5*y + 1
    return cas.vertcat(g1, g2)

In [27]:
X = cas.MX.sym('X', 2)

In [28]:
print(g(X))

@1=X[0], @2=X[1], vertcat(tanh(((((2+@1)*sq(@2))/25)-0.5)), (1+(sin(@1)-(0.5*@2))))


In [29]:
cas.jacobian(g(X),X)

MX(@1=X[0], @2=(2+@1), @3=X[1], @4=sq(@3), @5=(1-sq(tanh((((@2*@4)/25)-0.5)))), @6=0.04, ((zeros(2x2)[:4:2] = vertcat((@5*(@6*@4)), cos(@1)))[1:5:2] = vertcat((@5*(@6*(@2*(2.*@3)))), -0.5))')

In [30]:
Jf = cas.Function('Jf', [X], [cas.jacobian(g(X),X)])

In [31]:
Jf

Function(Jf:(i0[2])->(o0[2x2]) MXFunction)

In [32]:
x_0

array([-0.8,  2. ])

In [33]:
Jf(x_0)

DM(
[[0.145733, 0.174879], 
 [0.696707, -0.5]])

In [34]:
def delta_x(x):
    jac = Jf(x)
    return cas.solve(-jac, g(x))

In [35]:
def step (x):
    dx = delta_x(x)
    return np.array(x + dx)

In [36]:
x = x_0
for i in range(5):
    x = step(x)
    print(x)

[[0.61114966]
 [2.5316027 ]]
[[0.16714937]
 [2.42035621]]
[[0.19440912]
 [2.38650392]]
[[0.19453578]
 [2.38662219]]
[[0.19453578]
 [2.38662219]]


## Ejercicio 3

In [41]:
x = cas.MX.sym('x',2)
lam = cas.MX.sym('lam')
nu = cas.MX.sym('nu')

In [39]:
f = cas.Function('f', [x], [x[0]**2 + cas.tanh(x[1])**2])
g = cas.Function('g', [x], [cas.cos(x[0] + x[1]) + 0.5])
h = cas.Function('h', [x], [cas.sin(x[0]) + 0.5])

In [42]:
L = cas.Function('L', [x, lam, nu], [f(x) + lam * g(x) + nu * h(x)])

In [44]:
L([-0.5, -1.8], 2, 3)

DM(0.875613)

In [104]:
params = cas.Function('params', [x, lam, nu], [
    f(x), g(x), h(x),
    cas.gradient(f(x), x), cas.jacobian(g(x),x), cas.jacobian(h(x),x),
    cas.hessian(L(x, lam, nu),x)[0],
])

In [54]:
params([-0.5, -1.8], 0, 0)

(DM(1.14644),
 DM(-0.166276),
 DM(0.0205745),
 DM([-1, -0.196099]),
 DM([0.745705, 0.745705]),
 DM([0.877583, 0]),
 DM(
 [[2, 0], 
  [0, -0.349887]]))

In [118]:
hga = cas.Function(
    'HGAmatrices',
    [x, lam, nu,],
    [
        cas.hessian(L(x, lam, nu),x)[0],
        cas.gradient(f(x), x),
        cas.vertcat(cas.jacobian(g(x),x), cas.jacobian(h(x),x)),
        cas.horzcat(-g(x), -cas.inf),
        cas.horzcat(-g(x), - h(x))        
    ],
    ['x', 'lam', 'nu'],
    ['h', 'g', 'a', 'lba', 'uba'],
)

In [119]:
zzz = hga(x = [-0.5, -1.8], lam = 0, nu = 0)

In [127]:
zzz

{'a': DM(
 [[0.745705, 0.745705], 
  [0.877583, 00]]), 'g': DM([-1, -0.196099]), 'h': DM(
 [[2, 0], 
  [0, -0.349887]]), 'lba': DM([[0.166276, -inf]]), 'uba': DM([[0.166276, -0.0205745]])}

In [121]:
zzz['h'].sparsity().spy()

**
**


In [122]:
zzz['a'].sparsity().spy()

**
*.


In [128]:
qp_struct = {
    'a': zzz['a'].sparsity(),
    'h': zzz['h'].sparsity()
}

In [143]:
opts = {"print_iter": False}
solver = cas.conic('solver', 'qrqp', qp_struct, opts)

-------------------------------------------
This is casadi::QRQP
Number of variables:                               2
Number of constraints:                             2
Number of nonzeros in H:                           4
Number of nonzeros in A:                           3
Number of nonzeros in KKT:                        12
Number of nonzeros in QR(V):                      10
Number of nonzeros in QR(R):                      10


In [144]:
res = solver(**zzz)

In [137]:
res

{'cost': DM(-0.0349525),
 'lam_a': DM([0.378594, 0.871222]),
 'lam_x': DM([0, 0]),
 'x': DM([-0.0234445, 0.246423])}

In [148]:
x_p = cas.DM([-0.5, -1.8])
lam_p = 0
nu_p = 0
for ii in range(5):
    zzz = hga(x = x_p, lam = lam_p, nu = nu_p)
    res = solver(**zzz)
    x_p = x_p + res['x']
    print(x_p)
    lam_p = res['lam_a'][0]
    nu_p = res['lam_a'][1]

[-0.523444, -1.55358]
[-0.523599, -1.57071]
[-0.523599, -1.5708]
[-0.523599, -1.5708]
[-0.523599, -1.5708]


## Ejercicio 4: Golf

In [149]:
m = 0.04593
g = 9.81
c = 17.5e-5

In [150]:
p = cas.MX.sym('p', 2)
v = cas.MX.sym('v', 2)

In [154]:
rhs = cas.vertcat(
    v[0],
    v[1],
    -c*cas.norm_2(v)*v[0]/m,
    -g-c*cas.norm_2(v)*v[1]/m
)

In [155]:
f = cas.Function('rhs', [cas.vertcat(p,v)], [rhs])

In [156]:
f([0.0, 0.0, 35.0, 30.0])

DM([35, 30, -6.14737, -15.0792])

In [157]:
ode = {
    'x':cas.vertcat(p,v),
    'ode': rhs
}
options = {'tf':1}
intg = cas.integrator('intg', 'cvodes', ode, options)

In [158]:
intg

Function(intg:(x0[4],p[0],z0[0],rx0[0],rp[0],rz0[0])->(xf[4],qf[0],zf[0],rxf[0],rqf[0],rzf[0]) CvodesInterface)

In [159]:
intg(x0 = [0.0, 0.0, 35.0, 30.0])

{'qf': DM([]),
 'rqf': DM([]),
 'rxf': DM([]),
 'rzf': DM([]),
 'xf': DM([32.3549, 23.0661, 30.075, 16.6423]),
 'zf': DM([])}

In [170]:
intg(x0 = X0)['xf']

MX(intg(X0, 0x1, 0x1, 0x1, 0x1, 0x1){0})

In [171]:
X0 = cas.MX.sym('X0', 4)
fly1sec = cas.Function('fly1sec', [X0], [intg(x0 = X0)['xf']])

In [174]:
fly1sec([0.0, 0.0, 35.0, 20.0])

DM([32.6405, 13.9607, 30.5608, 8.26437])

In [175]:
T = cas.MX.sym('T')

In [181]:
rhs

MX(vertcat(v[0], v[1], (((-0.000175*||v||_F)*v[0])/0.04593), (-9.81-(((0.000175*||v||_F)*v[1])/0.04593))))

In [182]:
rhs = cas.vertcat(
    v[0],
    v[1],
    -c*cas.norm_2(v)*v[0]/m,
    -g-c*cas.norm_2(v)*v[1]/m
)
ode_T = {
    'x':cas.vertcat(p,v),
    'ode': T*rhs,
    'p':T
}
options = {'tf':1}
intg = cas.integrator('intg', 'cvodes', ode_T, options)

In [183]:
fly = cas.Function('fly', [X0, T], [intg(x0 = X0, p = T)['xf']])

In [190]:
fly([0.0, 0.0, 35.0, 30.0],5)

DM([130.338, 8.27205, 19.8961, -21.2868])

In [185]:
theta = cas.MX.sym('theta')
v = cas.MX.sym('v')

In [189]:
fly

Function(fly:(i0[4],i1)->(o0[4]) MXFunction)

In [194]:
theta_rad = theta/180*cas.pi

In [195]:
shoot = cas.Function('shoot',
                     [v, theta, T],
                     [fly(cas.vertcat(0.0, 0.0, v*cas.cos(theta_rad), v*cas.sin(theta_rad)),T)]
                    )

In [196]:
shoot(50,30,5)

DM([155.243, -11.0833, 22.6012, -23.8282])

In [198]:
x = shoot(50,30,T)
height = x[1]
 
rf = cas.rootfinder('rf','newton',{'x':T,'g':height})
res = rf(x0=5)

In [199]:
res

{'x': DM(4.49773)}

In [205]:
x = shoot(v,theta,T)
height = x[1]
 
rf = cas.rootfinder('rf','newton',{'x':T,'p':cas.vertcat(v,theta),'g':height})
res = rf(x0=5,p=cas.vertcat(v,theta))
 
T_landing = res["x"]
xf = shoot(v,theta,T_landing)
 
shoot_distance = cas.Function('shoot_distance',[v,theta],[xf[0]])
 
print(shoot_distance(50,40))

152.824
